# 第5章: 出力の書式設定とClaude の代弁

- [レッスン](#lesson)
- [演習](#exercises)
- [例題プレイグラウンド](#example-playground)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立してください。

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
import json

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt, system='', prefill=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages":[
              {"role": "user", "content": prompt},
              {"role": "assistant", "content": prefill}
            ],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## レッスン

**Claudeは様々な方法で出力を書式設定することができます**。必要なのは、そうするように依頼することだけです！

その方法の一つは、XMLタグを使用して、レスポンスを他の余分なテキストから分離することです。XMLタグを使ってプロンプトをより明確でClaude にとって解析しやすくできることは既に学びました。実は、**XMLタグを使ってClaude の出力をより明確で人間にとって理解しやすくする**ことも可能なのです。

### 例

第2章で解決した「詩の前文問題」を覚えていますか？あの時は前文を完全にスキップするようにClaude に依頼することで解決しました。実は、**Claude に詩をXMLタグで囲むように指示する**ことでも同様の結果を得ることができます。

In [ ]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

なぜこのようなことをしたいのでしょうか？出力を**XMLタグで囲むことで、エンドユーザーがXMLタグ間のコンテンツを抽出する短いプログラムを書くことで、確実に詩だけを、詩のみを取得できる**からです。

このテクニックの拡張版は、**最初のXMLタグを`assistant`ターンに置くこと**です。`assistant`ターンにテキストを置くと、基本的にClaude が既に何かを言ったことを伝え、そこから続けるべきであることを指示します。このテクニックは「Claude の代弁」または「Claude のレスポンスの事前入力」と呼ばれます。

以下では、最初の`<haiku>`XMLタグでこれを行っています。Claude が私たちが残した場所から直接続けていることに注目してください。

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Claude は他の出力書式設定スタイル、特に`JSON`の使用も得意です。JSON出力を強制したい場合（決定論的ではありませんが、それに近い）、Claude のレスポンスを開始括弧`{`で事前入力することもできます。

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

以下は、**同一プロンプト内での複数の入力変数と出力書式設定指定の例で、すべてXMLタグを使用して行われています**。

In [ ]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

#### ボーナスレッスン

API経由でClaude を呼び出す場合、閉じるXMLタグを`stop_sequences`パラメータに渡すことで、Claude が所望のタグを出力したらサンプリングを停止させることができます。これにより、Claude が既に気にしている答えを提供した後の結論的なコメントを排除し、コストと最終トークンまでの時間を節約できます。

上記の内容を変更せずにレッスンのプロンプトを試したい場合は、レッスンノートブックの一番下までスクロールして[**例題プレイグラウンド**](#example-playground)をご覧ください。

---

## 演習
- [演習 5.1 - Steph Curry GOAT](#exercise-51---steph-curry-goat)
- [演習 5.2 - 2つの俳句](#exercise-52---two-haikus)
- [演習 5.3 - 2つの俳句、2つの動物](#exercise-53---two-haikus-two-animals)

### 演習 5.1 - Steph Curry GOAT
選択を強制されると、Claude はMichael Jordanを史上最高のバスケットボール選手に指名します。Claude に他の選手を選ばせることはできるでしょうか？

`PREFILL`変数を変更して、**Claude に史上最高のバスケットボール選手はStephen Curryであるという詳細な論証をさせてください**。この演習の焦点なので、`PREFILL`以外は変更しないようにしてください。

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."

# Prefill for Claude's response
PREFILL = ""

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、下のセルを実行してください！

In [ ]:
print(hints.exercise_5_1_hint)

### 演習 5.2 - 2つの俳句
XMLタグを使用して以下の`PROMPT`を修正し、Claude が動物について1つではなく2つの俳句を書くようにしてください。1つの詩が終わり、もう1つが始まることが明確になるようにしてください。

In [ ]:
# Variable content
ANIMAL = "cats"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、下のセルを実行してください！

In [ ]:
print(hints.exercise_5_2_hint)

### 演習 5.3 - 2つの俳句、2つの動物
**Claude に2つの異なる動物について2つの俳句を作らせる**ように以下の`PROMPT`を修正してください。最初の置換の代替として`{ANIMAL1}`を、2番目の置換の代替として`{ANIMAL2}`を使用してください。

In [ ]:
# First input variable
ANIMAL1 = "Cat"

# Second input variable
ANIMAL2 = "Dog"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL1}. Put it in <haiku> tags."

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、下のセルを実行してください！

In [ ]:
print(hints.exercise_5_3_hint)

### おめでとうございます！

ここまでのすべての演習を解けた場合、次の章に進む準備ができています。楽しいプロンプト作成を！

---

## 例題プレイグラウンド

これは、このレッスンで紹介されたプロンプト例を自由に試し、プロンプトを調整してClaude のレスポンスにどのような影響を与えるかを確認するエリアです。

In [ ]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))